In [3]:
import numpy as np
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [5]:
# Load Data (replace this with actual CSV file or DataFrame)
data = pd.read_csv('merged_properties.csv')  # Replace 'your_file.csv' with your actual file path
# Features and targets
# X = data.drop(columns=['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']).values
X = data[['vf', 'c55e', 'e15e', 'q15e', 'ϵ11e', 'μ11e', 'α11e', 'ρe', 'c55f', 'e15f', 'q15f', 'ϵ11f', 'μ11f', 'α11f', 'ρf']].values
y = data[['c44', 'e15', 'q15', 'μ11', 'ϵ11', 'α11', 'ρ']].values

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Split first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scalers_X = []
X_train_scaled = np.zeros_like(X_train)
X_test_scaled = np.zeros_like(X_test)

for i in range(X.shape[1]):
    scaler = MinMaxScaler()
    X_train_scaled[:, i] = scaler.fit_transform(X_train[:, i].reshape(-1, 1)).flatten()
    X_test_scaled[:, i] = scaler.transform(X_test[:, i].reshape(-1, 1)).flatten()
    scalers_X.append(scaler)

# Scale y column-wise
scalers_y = []
y_train_scaled = np.zeros_like(y_train)
y_test_scaled = np.zeros_like(y_test)

for i in range(y.shape[1]):
    scaler = MinMaxScaler()
    y_train_scaled[:, i] = scaler.fit_transform(y_train[:, i].reshape(-1, 1)).flatten()
    y_test_scaled[:, i] = scaler.transform(y_test[:, i].reshape(-1, 1)).flatten()
    scalers_y.append(scaler)


Shape of X: (1045, 15)
Shape of y: (1045, 7)


In [12]:

# Load data (Assume X, y are already loaded)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling
scaler_X, scaler_y = StandardScaler(), StandardScaler()
X_train_scaled, X_test_scaled = scaler_X.fit_transform(X_train), scaler_X.transform(X_test)
y_train_scaled, y_test_scaled = scaler_y.fit_transform(y_train), scaler_y.transform(y_test)

# Ridge Regression
ridge = MultiOutputRegressor(Ridge(alpha=1.0)).fit(X_train_scaled, y_train_scaled)
y_pred_ridge = scaler_y.inverse_transform(ridge.predict(X_test_scaled))

# Elastic Net
elastic_net = MultiOutputRegressor(ElasticNet(alpha=1.0, l1_ratio=0.5)).fit(X_train_scaled, y_train_scaled)
y_pred_elastic = scaler_y.inverse_transform(elastic_net.predict(X_test_scaled))

# Evaluation
ridge_mae = mean_absolute_error(y_test, y_pred_ridge)
elastic_mae = mean_absolute_error(y_test, y_pred_elastic)

print(f"Ridge MAE: {ridge_mae:.4f}")
print(f"Elastic Net MAE: {elastic_mae:.4f}")

Ridge MAE: 8085212901.9415
Elastic Net MAE: 11471384065.1735


In [11]:
y_pred_scaled = ridge.predict(X_test_scaled)

y_pred = scaler_y.inverse_transform(y_pred_scaled)

np.set_printoptions(linewidth=np.inf)  # Set the print options to avoid line breaks
for i in range(10):
    print(f"Predicted: {y_pred[i]}")
    print(f"Actual:    {y_test[i]}")
    print("-" * 50)

Predicted: [ 6.57297489e+10  7.57492483e+00  3.22094676e+02 -3.39824954e-04  1.10836199e-01 -6.88994820e-10  6.82178781e+03]
Actual:    [ 5.75659234e+10  3.79152653e-02  1.01572435e+02  3.00662118e-05  4.87692269e-10 -6.17503512e-13  7.33000000e+03]
--------------------------------------------------
Predicted: [ 1.77487235e+11  1.60687483e-01  2.89085891e+01  1.36927952e-03  3.57815946e-02 -6.42206148e-09  6.40646991e+03]
Actual:    [9.65571615e+10 5.65677431e-01 1.46347462e-03 7.18733615e-04 7.40538530e-10 2.87408973e-10 7.10000000e+03]
--------------------------------------------------
Predicted: [ 1.11068591e+10 -9.61700403e-01  2.45374597e+01  2.33363513e-03 -2.62243003e-03  1.12290208e-09  4.96763542e+03]
Actual:    [3.06869106e+10 3.47867300e-04 1.45706883e-06 3.18686247e-03 1.43839504e-10 7.08191652e-10 5.28000000e+03]
--------------------------------------------------
Predicted: [ 3.46876925e+10  1.42984631e+01 -3.00070021e+01 -4.24178273e-04 -5.55018429e-02 -2.36000554e-08  8.

In [13]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Define and train the model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)

print(f"XGBoost MAE: {mae:.4f}")

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/opt/anaconda3/envs/Thesis/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <54A1AE05-1E14-3DA2-A8D0-062134694298> /opt/anaconda3/envs/Thesis/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no 